In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("paramaggarwal/fashion-product-images-dataset")

print("Path to dataset files:", path)

Path to dataset files: /teamspace/studios/this_studio/.cache/kagglehub/datasets/paramaggarwal/fashion-product-images-dataset/versions/1


In [1]:
!pip install kaggle
!pip install sentence-transformers
!pip install torch
!pip install tqdm
!conda install -c pytorch faiss-gpu

Channels:
 - pytorch
 - defaults
Platform: linux-64
Solving environment: done

# All requested packages already installed.



In [2]:
import os
import torch
from glob import glob
from PIL import Image
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
import pandas as pd
import json
from tqdm import tqdm

In [10]:
def load_image(image_path):
    try:
        img = Image.open(image_path).convert('RGB')
        return img
    except:
        print(f"Error loading image {image_path}")
        return None

def make_df(filepath):
    return pd.read_csv(filepath, on_bad_lines='skip')

def add_imgPaths(df, img_folder):
    df['img_path'] = df['id'].apply(
        lambda x: os.path.join(img_folder, f"{x}.jpg")
    )



In [5]:
# /teamspace/studios/this_studio/.cache/kagglehub/datasets/paramaggarwal/fashion-product-images-dataset/versions/1/fashion-dataset/styles.csv
csv_filepath = "/teamspace/studios/this_studio/.cache/kagglehub/datasets/paramaggarwal/fashion-product-images-dataset/versions/1/fashion-dataset/styles.csv"
img_folder = "/teamspace/studios/this_studio/.cache/kagglehub/datasets/paramaggarwal/fashion-product-images-dataset/versions/1/fashion-dataset/images"

dfs  = make_df(csv_filepath)
add_imgPaths(dfs, img_folder)

In [6]:
def store_embeddings_json(df, model, batch_size, output_index, output_json):
    index = None
    all_product_info = {}
    for bt in tqdm(range(0, len(df), batch_size)):
        imgs = []     
        id_product_info = {} 
        for i in tqdm(range(bt, min(bt + batch_size, len(df)))):
            row = df.iloc[i]
            img = load_image(row['img_path'])
            if img is  None:
                continue
        
            imgs.append(img)
            id_product_info[i] = {
                "gender" : row['gender'],
                "masterCategory" : row['masterCategory'],
                "subCategory" : row['subCategory'],
                "articleType" : row['articleType'],
                "baseColour" : row['baseColour'],
                "season" : row['season'],
                "usage" : row['usage'],
                "productDisplayName" : row['productDisplayName']
            }

            
        batch_embedding = model.encode(
            imgs, 
            batch_size=batch_size,
            convert_to_numpy=True,
        )
        
        embeddings = np.array(batch_embedding).astype('float32')

        if index is None:
            dimension = embeddings.shape[1]
            index = faiss.IndexIDMap(faiss.IndexFlatIP(dimension))
            
        ids = np.array(list(id_product_info.keys()), dtype=np.int64)
        all_product_info.update(id_product_info)
        index.add_with_ids(embeddings, ids)

        
        
    with open(output_json, 'w') as f:
        json.dump(all_product_info, f)


    faiss.write_index(index, output_index)
    print("Embeddings and product info stored successfully.")
    return 


In [3]:
batch_size = 256
model = SentenceTransformer('clip-ViT-B-32', device="cuda")

Loading weights:   0%|          | 0/398 [00:00<?, ?it/s]

CLIPModel LOAD REPORT from: /teamspace/studios/this_studio/.cache/huggingface/hub/models--sentence-transformers--clip-ViT-B-32/snapshots/327ab6726d33c0e22f920c83f2ff9e4bd38ca37f/0_CLIPModel
Key                                  | Status     |  | 
-------------------------------------+------------+--+-
vision_model.embeddings.position_ids | UNEXPECTED |  | 
text_model.embeddings.position_ids   | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
The image processor of type `CLIPImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 


In [8]:
print(dfs.shape)

(44424, 11)


In [11]:
output_path_index = './fashion_product_images.index'
output_path_json = './fashion_product_info.json'
store_embeddings_json(dfs, model, batch_size, output_path_index, output_path_json)

 15%|█▍        | 26/174 [02:25<13:49,  5.61s/it]

Error loading image /teamspace/studios/this_studio/.cache/kagglehub/datasets/paramaggarwal/fashion-product-images-dataset/versions/1/fashion-dataset/images/39403.jpg


 36%|███▌      | 63/174 [06:11<11:27,  6.20s/it]

Error loading image /teamspace/studios/this_studio/.cache/kagglehub/datasets/paramaggarwal/fashion-product-images-dataset/versions/1/fashion-dataset/images/39410.jpg


 72%|███████▏  | 126/174 [12:28<05:00,  6.26s/it]

...

Error loading image /teamspace/studios/this_studio/.cache/kagglehub/datasets/paramaggarwal/fashion-product-images-dataset/versions/1/fashion-dataset/images/39425.jpg


 90%|████████▉ | 156/174 [15:35<01:55,  6.41s/it]

Error loading image /teamspace/studios/this_studio/.cache/kagglehub/datasets/paramaggarwal/fashion-product-images-dataset/versions/1/fashion-dataset/images/12347.jpg


100%|██████████| 174/174 [17:19<00:00,  5.97s/it]


Embeddings and product info stored successfully.


In [9]:
def retrieve_similar_images(query, model, index, info, top_k=3):

    if query.endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
        query = Image.open(query)

    query_features = model.encode(query)
    query_features = query_features.astype(np.float32).reshape(1, -1)

    distances, indices = index.search(query_features, top_k)

    retrieved_info = [
        info[str(int(idx))]
        for idx in indices[0]
        if idx != -1
    ]

    return query, retrieved_info

In [5]:
output_path_index = './fashion_product_images.index'
output_path_json = './fashion_product_info.json'
def load_faiss_index(index_path,json_path):
    index = faiss.read_index(index_path)
    with open(json_path, 'r') as f:
        info = json.load(f)
    
    return index, info

index, info = load_faiss_index(output_path_index, output_path_json)

In [11]:
query = '/teamspace/studios/this_studio/image.png'
query, retrieved_info = retrieve_similar_images(query, model, index, info, top_k=3)

In [12]:
print(retrieved_info)

[{'gender': 'Men', 'masterCategory': 'Footwear', 'subCategory': 'Shoes', 'articleType': 'Casual Shoes', 'baseColour': 'Black', 'season': 'Winter', 'usage': 'Casual', 'productDisplayName': 'Converse Men Black Shoes'}, {'gender': 'Men', 'masterCategory': 'Footwear', 'subCategory': 'Shoes', 'articleType': 'Casual Shoes', 'baseColour': 'Black', 'season': 'Summer', 'usage': 'Casual', 'productDisplayName': "Converse Men's AS Canvas HI Black Shoe"}, {'gender': 'Men', 'masterCategory': 'Footwear', 'subCategory': 'Shoes', 'articleType': 'Casual Shoes', 'baseColour': 'Black', 'season': 'Fall', 'usage': 'Casual', 'productDisplayName': "Numero Uno Men's Casual Black Canvas Shoe"}]
